In [1]:
import os
import argparse
import abagen
import pandas as pd

In [3]:
outdir = 'data/microarray/'

In [5]:
expression, coords = abagen.get_samples_in_mask(mask=None, data_dir = outdir, donors = 'all')

In [6]:
type(expression)

pandas.core.frame.DataFrame

In [7]:
expression.shape

(3466, 15633)

In [11]:
type(coords)

pandas.core.frame.DataFrame

In [13]:
coords.head()

,x,y,z
well_id,,,
594,6.281693,-26.769456,46.030920
2985,29.463218,17.371072,-6.841868
2801,30.916352,-19.971094,12.528056
2273,-24.471659,-23.995584,-2.006094
2785,33.052748,-30.720132,-12.020580


In [14]:
from datatable import fread

In [247]:
tmp = fread('data/HumanExpressionMatrix_samples_pipeline_v1.csv', header = True).to_pandas()

In [46]:
tmp.shape

(15125, 3703)

In [249]:
tmp['Gene']

0              61E3.4
1        A_24_P614148
2                A1BG
3              A26C1B
4               A2BP1
             ...     
15120            ZXDC
15121          ZYG11B
15122             ZYX
15123           ZZEF1
15124            ZZZ3
Name: Gene, Length: 15125, dtype: object

In [35]:
import pandas as pd

from re import search

In [198]:
donors

,donorID,donorFileID
0,H0351.2001,donor9861
1,H0351.2002,donor10021
2,H0351.1009,donor12876
3,H0351.1012,donor14380
4,H0351.1015,donor15496
5,H0351.1016,donor15697


In [218]:
donors = pd.read_csv('data/donors.csv')
sample_data = []
for index, row in donors.iterrows():
    
    donor_dir = 'normalized_microarray_{}'.format(row['donorFileID'])
    samplefile = os.path.join(outdir, donor_dir, 'SampleAnnot.csv')
    df_sample = pd.read_csv(samplefile)
    
    sample_ids = df_sample.apply(lambda x: '%s-%s-%s' % (x['structure_id'], x['slab_num'], x['well_id']), 
                                 axis = 1)
    df_sample['SampleID'] = sample_ids
    
    df_sample['Donor'] = row['donorID']
    
    sample_data.append(df_sample)
    
df_samples = pd.concat(sample_data, axis = 0, ignore_index = True)
df_samples = df_samples.loc[df_samples['well_id'].isin(expression.index)]

In [161]:
expression.head()

gene_symbol,A1BG,A1BG-AS1,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADAT,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
well_id,,,,,,,,,,,,,,,,,,,,,
594,0.301206,0.633459,0.524247,0.418119,0.354588,0.264215,0.747265,0.896750,0.267889,0.854105,...,0.664240,0.425123,0.393411,0.539312,0.441774,0.344128,0.790665,0.750759,0.548501,0.335783
2985,0.789466,0.423733,0.371010,0.236624,0.935364,0.710047,0.437137,0.661248,0.459832,0.432018,...,0.091802,0.756656,0.240631,0.915214,0.304052,0.508392,0.569169,0.770941,0.568334,0.889131
2801,0.950836,0.356655,0.597191,0.041786,0.607998,0.954603,0.204959,0.408170,0.408236,0.250875,...,0.620293,0.678288,1.000000,0.136651,0.060707,0.880952,0.438016,0.549799,0.148812,0.406152
2273,0.255668,0.211723,0.781888,0.370243,0.620974,0.137888,0.034473,0.068002,0.885394,0.148016,...,0.240219,0.131628,0.321255,0.005255,0.007859,0.908355,0.051453,0.286382,0.527082,0.017642
2785,0.282351,0.284394,0.859964,0.132408,0.454795,0.880341,0.479368,0.065681,0.682845,0.422405,...,0.402900,0.326117,0.457908,0.446117,0.031938,0.630605,0.132883,0.134859,0.263702,0.678706


In [212]:
df_sample_ids = df_samples.loc[:,['well_id', 'SampleID']]

In [213]:
expression_new = (expression
 .merge(df_sample_ids, on = 'well_id', how = 'left')
 .drop(labels = 'well_id', axis = 1))

In [219]:
df_samples = df_samples.set_index('SampleID').reindex(index = expression_new['SampleID']).reset_index(level = 'SampleID')


In [220]:
df_samples

,SampleID,structure_id,slab_num,well_id,slab_type,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,Donor
0,4077-22-594,4077,22,594,CX,PCLa-i,"paracentral lobule, anterior part, right, infe...",37470,87,52,116,5.9,-27.7,49.7,H0351.2001
1,4323-11-2985,4323,11,2985,CX,Cl,"claustrum, right",40517,66,92,63,29.2,17.0,-2.9,H0351.2001
2,4323-18-2801,4323,18,2801,CX,Cl,"claustrum, right",41516,66,81,104,28.2,-22.8,16.8,H0351.2001
3,4440-18-2273,4440,18,2273,CX,LGd,"dorsal lateral geniculate nucleus, left",41473,116,94,101,-24.6,-24.6,1.3,H0351.2001
4,4266-17-2785,4266,17,2785,CX,CA4,"CA4 field, right",41142,63,104,106,31.1,-31.3,-7.3,H0351.2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461,4342-7-159439039,4342,7,159439039,CX,BLA,"basolateral nucleus, left",147057074,112,127,94,-21.0,-3.0,-18.0,H0351.1016
3462,4785-64-159438991,4785,64,159438991,CB,Glo,"globose nucleus, left",156978055,93,139,156,-2.0,-65.0,-30.0,H0351.1016
3463,4783-64-159438975,4783,64,159438975,CB,Emb,"emboliform nucleus, left",156978968,93,136,153,-2.0,-62.0,-27.0,H0351.1016
3464,4782-64-159438959,4782,64,159438959,CB,Dt,"dentate nucleus, left",156419141,93,127,155,-2.0,-64.0,-18.0,H0351.1016


In [244]:
expression_new_1 = expression_new.set_index('SampleID')
expression_new_1

,A1BG,A1BG-AS1,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADAT,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
SampleID,,,,,,,,,,,,,,,,,,,,,
4077-22-594,0.301206,0.633459,0.524247,0.418119,0.354588,0.264215,0.747265,0.896750,0.267889,0.854105,...,0.664240,0.425123,0.393411,0.539312,0.441774,0.344128,0.790665,0.750759,0.548501,0.335783
4323-11-2985,0.789466,0.423733,0.371010,0.236624,0.935364,0.710047,0.437137,0.661248,0.459832,0.432018,...,0.091802,0.756656,0.240631,0.915214,0.304052,0.508392,0.569169,0.770941,0.568334,0.889131
4323-18-2801,0.950836,0.356655,0.597191,0.041786,0.607998,0.954603,0.204959,0.408170,0.408236,0.250875,...,0.620293,0.678288,1.000000,0.136651,0.060707,0.880952,0.438016,0.549799,0.148812,0.406152
4440-18-2273,0.255668,0.211723,0.781888,0.370243,0.620974,0.137888,0.034473,0.068002,0.885394,0.148016,...,0.240219,0.131628,0.321255,0.005255,0.007859,0.908355,0.051453,0.286382,0.527082,0.017642
4266-17-2785,0.282351,0.284394,0.859964,0.132408,0.454795,0.880341,0.479368,0.065681,0.682845,0.422405,...,0.402900,0.326117,0.457908,0.446117,0.031938,0.630605,0.132883,0.134859,0.263702,0.678706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4342-7-159439039,0.850792,0.378878,0.595801,0.925642,0.210548,0.858887,0.239078,0.713788,0.920553,0.541071,...,0.085105,0.935246,0.563675,0.736405,0.388664,0.447696,0.232133,0.300047,0.547204,0.838614
4785-64-159438991,0.634737,0.011386,0.956441,1.000000,1.000000,0.664351,0.365788,0.161670,0.899774,0.319224,...,0.423026,0.016783,0.084266,0.700463,0.878950,0.480929,0.048979,0.342996,0.698855,0.910304
4783-64-159438975,0.888546,0.061438,0.946679,0.227138,0.915732,0.908007,0.842345,0.185598,0.993214,0.266372,...,0.173606,0.172717,0.085276,0.193787,0.788015,0.898143,0.000242,0.231036,0.867496,0.776488


In [245]:
expression_new_1.index.name = None
expression_new_1

,A1BG,A1BG-AS1,A2M,A2ML1,A3GALT2,A4GALT,AAAS,AACS,AADACL3,AADAT,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
4077-22-594,0.301206,0.633459,0.524247,0.418119,0.354588,0.264215,0.747265,0.896750,0.267889,0.854105,...,0.664240,0.425123,0.393411,0.539312,0.441774,0.344128,0.790665,0.750759,0.548501,0.335783
4323-11-2985,0.789466,0.423733,0.371010,0.236624,0.935364,0.710047,0.437137,0.661248,0.459832,0.432018,...,0.091802,0.756656,0.240631,0.915214,0.304052,0.508392,0.569169,0.770941,0.568334,0.889131
4323-18-2801,0.950836,0.356655,0.597191,0.041786,0.607998,0.954603,0.204959,0.408170,0.408236,0.250875,...,0.620293,0.678288,1.000000,0.136651,0.060707,0.880952,0.438016,0.549799,0.148812,0.406152
4440-18-2273,0.255668,0.211723,0.781888,0.370243,0.620974,0.137888,0.034473,0.068002,0.885394,0.148016,...,0.240219,0.131628,0.321255,0.005255,0.007859,0.908355,0.051453,0.286382,0.527082,0.017642
4266-17-2785,0.282351,0.284394,0.859964,0.132408,0.454795,0.880341,0.479368,0.065681,0.682845,0.422405,...,0.402900,0.326117,0.457908,0.446117,0.031938,0.630605,0.132883,0.134859,0.263702,0.678706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4342-7-159439039,0.850792,0.378878,0.595801,0.925642,0.210548,0.858887,0.239078,0.713788,0.920553,0.541071,...,0.085105,0.935246,0.563675,0.736405,0.388664,0.447696,0.232133,0.300047,0.547204,0.838614
4785-64-159438991,0.634737,0.011386,0.956441,1.000000,1.000000,0.664351,0.365788,0.161670,0.899774,0.319224,...,0.423026,0.016783,0.084266,0.700463,0.878950,0.480929,0.048979,0.342996,0.698855,0.910304
4783-64-159438975,0.888546,0.061438,0.946679,0.227138,0.915732,0.908007,0.842345,0.185598,0.993214,0.266372,...,0.173606,0.172717,0.085276,0.193787,0.788015,0.898143,0.000242,0.231036,0.867496,0.776488
4782-64-159438959,0.937625,0.024379,0.980379,0.177114,0.991332,0.825149,0.515826,0.181082,0.512344,0.242297,...,0.608362,0.826495,0.164309,0.783844,0.330140,0.152788,0.038240,0.286003,0.571000,0.696873


In [246]:
expression_new_2 = expression_new_1.transpose()
expression_new_2

,4077-22-594,4323-11-2985,4323-18-2801,4440-18-2273,4266-17-2785,4267-17-2793,4782-72-2795,4788-67-2874,4251-17-2858,4254-17-2850,...,4329-7-159438945,4308-7-159439119,4379-7-159439103,4367-7-159439087,4360-7-159439071,4342-7-159439039,4785-64-159438991,4783-64-159438975,4782-64-159438959,4782-63-159438943
A1BG,0.301206,0.789466,0.950836,0.255668,0.282351,0.320764,0.706279,0.958863,0.839536,0.654914,...,0.832291,0.626030,0.743731,0.895883,0.908079,0.850792,0.634737,0.888546,0.937625,0.398458
A1BG-AS1,0.633459,0.423733,0.356655,0.211723,0.284394,0.286426,0.341985,0.268927,0.322783,0.234039,...,0.223054,0.194303,0.292847,0.279058,0.590386,0.378878,0.011386,0.061438,0.024379,0.055432
A2M,0.524247,0.371010,0.597191,0.781888,0.859964,0.166605,0.919367,0.860522,0.410158,0.283280,...,0.733399,0.066039,0.487398,0.700757,0.032677,0.595801,0.956441,0.946679,0.980379,0.963082
A2ML1,0.418119,0.236624,0.041786,0.370243,0.132408,0.643291,0.378278,0.050163,0.544712,0.117604,...,0.967032,0.681433,0.872861,0.677861,0.405917,0.925642,1.000000,0.227138,0.177114,0.848094
A3GALT2,0.354588,0.935364,0.607998,0.620974,0.454795,0.625648,0.434492,0.389345,0.817776,0.121842,...,0.978210,0.275538,0.952947,0.955870,0.801143,0.210548,1.000000,0.915732,0.991332,0.870341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.344128,0.508392,0.880952,0.908355,0.630605,0.859783,0.396860,0.541759,0.575857,0.216595,...,0.250133,0.896565,0.235778,0.462675,0.630409,0.447696,0.480929,0.898143,0.152788,0.712602
ZYG11B,0.790665,0.569169,0.438016,0.051453,0.132883,0.193244,0.452065,0.175061,0.332150,0.192049,...,0.038768,0.097552,0.061708,0.045848,0.187290,0.232133,0.048979,0.000242,0.038240,0.109018
ZYX,0.750759,0.770941,0.549799,0.286382,0.134859,0.587742,0.703823,0.783322,0.125455,0.020379,...,0.156711,0.076505,0.203059,0.170043,0.088419,0.300047,0.342996,0.231036,0.286003,0.569135
ZZEF1,0.548501,0.568334,0.148812,0.527082,0.263702,0.921318,0.665425,0.423761,0.277394,0.228080,...,0.061254,0.820702,0.141384,0.062722,0.144851,0.547204,0.698855,0.867496,0.571000,0.912293


In [250]:
expression_new_2.sort_index()

,4077-22-594,4323-11-2985,4323-18-2801,4440-18-2273,4266-17-2785,4267-17-2793,4782-72-2795,4788-67-2874,4251-17-2858,4254-17-2850,...,4329-7-159438945,4308-7-159439119,4379-7-159439103,4367-7-159439087,4360-7-159439071,4342-7-159439039,4785-64-159438991,4783-64-159438975,4782-64-159438959,4782-63-159438943
A1BG,0.301206,0.789466,0.950836,0.255668,0.282351,0.320764,0.706279,0.958863,0.839536,0.654914,...,0.832291,0.626030,0.743731,0.895883,0.908079,0.850792,0.634737,0.888546,0.937625,0.398458
A1BG-AS1,0.633459,0.423733,0.356655,0.211723,0.284394,0.286426,0.341985,0.268927,0.322783,0.234039,...,0.223054,0.194303,0.292847,0.279058,0.590386,0.378878,0.011386,0.061438,0.024379,0.055432
A2M,0.524247,0.371010,0.597191,0.781888,0.859964,0.166605,0.919367,0.860522,0.410158,0.283280,...,0.733399,0.066039,0.487398,0.700757,0.032677,0.595801,0.956441,0.946679,0.980379,0.963082
A2ML1,0.418119,0.236624,0.041786,0.370243,0.132408,0.643291,0.378278,0.050163,0.544712,0.117604,...,0.967032,0.681433,0.872861,0.677861,0.405917,0.925642,1.000000,0.227138,0.177114,0.848094
A3GALT2,0.354588,0.935364,0.607998,0.620974,0.454795,0.625648,0.434492,0.389345,0.817776,0.121842,...,0.978210,0.275538,0.952947,0.955870,0.801143,0.210548,1.000000,0.915732,0.991332,0.870341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.344128,0.508392,0.880952,0.908355,0.630605,0.859783,0.396860,0.541759,0.575857,0.216595,...,0.250133,0.896565,0.235778,0.462675,0.630409,0.447696,0.480929,0.898143,0.152788,0.712602
ZYG11B,0.790665,0.569169,0.438016,0.051453,0.132883,0.193244,0.452065,0.175061,0.332150,0.192049,...,0.038768,0.097552,0.061708,0.045848,0.187290,0.232133,0.048979,0.000242,0.038240,0.109018
ZYX,0.750759,0.770941,0.549799,0.286382,0.134859,0.587742,0.703823,0.783322,0.125455,0.020379,...,0.156711,0.076505,0.203059,0.170043,0.088419,0.300047,0.342996,0.231036,0.286003,0.569135
ZZEF1,0.548501,0.568334,0.148812,0.527082,0.263702,0.921318,0.665425,0.423761,0.277394,0.228080,...,0.061254,0.820702,0.141384,0.062722,0.144851,0.547204,0.698855,0.867496,0.571000,0.912293


In [251]:
expression_new_2.shape

(15633, 3466)

In [240]:
expression_new_2 = expression_new.set_index('SampleID').transpose()
expression_new_2.index.name = 'Gene'

expression_new_2.reset_index(level = 'Gene')

SampleID,Gene,4077-22-594,4323-11-2985,4323-18-2801,4440-18-2273,4266-17-2785,4267-17-2793,4782-72-2795,4788-67-2874,4251-17-2858,...,4329-7-159438945,4308-7-159439119,4379-7-159439103,4367-7-159439087,4360-7-159439071,4342-7-159439039,4785-64-159438991,4783-64-159438975,4782-64-159438959,4782-63-159438943
0,A1BG,0.301206,0.789466,0.950836,0.255668,0.282351,0.320764,0.706279,0.958863,0.839536,...,0.832291,0.626030,0.743731,0.895883,0.908079,0.850792,0.634737,0.888546,0.937625,0.398458
1,A1BG-AS1,0.633459,0.423733,0.356655,0.211723,0.284394,0.286426,0.341985,0.268927,0.322783,...,0.223054,0.194303,0.292847,0.279058,0.590386,0.378878,0.011386,0.061438,0.024379,0.055432
2,A2M,0.524247,0.371010,0.597191,0.781888,0.859964,0.166605,0.919367,0.860522,0.410158,...,0.733399,0.066039,0.487398,0.700757,0.032677,0.595801,0.956441,0.946679,0.980379,0.963082
3,A2ML1,0.418119,0.236624,0.041786,0.370243,0.132408,0.643291,0.378278,0.050163,0.544712,...,0.967032,0.681433,0.872861,0.677861,0.405917,0.925642,1.000000,0.227138,0.177114,0.848094
4,A3GALT2,0.354588,0.935364,0.607998,0.620974,0.454795,0.625648,0.434492,0.389345,0.817776,...,0.978210,0.275538,0.952947,0.955870,0.801143,0.210548,1.000000,0.915732,0.991332,0.870341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15628,ZXDC,0.344128,0.508392,0.880952,0.908355,0.630605,0.859783,0.396860,0.541759,0.575857,...,0.250133,0.896565,0.235778,0.462675,0.630409,0.447696,0.480929,0.898143,0.152788,0.712602
15629,ZYG11B,0.790665,0.569169,0.438016,0.051453,0.132883,0.193244,0.452065,0.175061,0.332150,...,0.038768,0.097552,0.061708,0.045848,0.187290,0.232133,0.048979,0.000242,0.038240,0.109018
15630,ZYX,0.750759,0.770941,0.549799,0.286382,0.134859,0.587742,0.703823,0.783322,0.125455,...,0.156711,0.076505,0.203059,0.170043,0.088419,0.300047,0.342996,0.231036,0.286003,0.569135
15631,ZZEF1,0.548501,0.568334,0.148812,0.527082,0.263702,0.921318,0.665425,0.423761,0.277394,...,0.061254,0.820702,0.141384,0.062722,0.144851,0.547204,0.698855,0.867496,0.571000,0.912293


In [252]:
expression = fread('data/HumanExpressionMatrix_samples_pipeline_abagen.csv', header = True).to_pandas()

In [254]:
expression.shape

(15633, 3467)

In [255]:
expression.head()

,Gene,4077-22-594,4323-11-2985,4323-18-2801,4440-18-2273,4266-17-2785,4267-17-2793,4782-72-2795,4788-67-2874,4251-17-2858,...,4329-7-159438945,4308-7-159439119,4379-7-159439103,4367-7-159439087,4360-7-159439071,4342-7-159439039,4785-64-159438991,4783-64-159438975,4782-64-159438959,4782-63-159438943
0,A1BG,0.301206,0.789466,0.950836,0.255668,0.282351,0.320764,0.706279,0.958863,0.839536,...,0.832291,0.626030,0.743731,0.895883,0.908079,0.850792,0.634737,0.888546,0.937625,0.398458
1,A1BG-AS1,0.633459,0.423733,0.356655,0.211723,0.284394,0.286426,0.341985,0.268927,0.322783,...,0.223054,0.194303,0.292847,0.279058,0.590386,0.378878,0.011386,0.061438,0.024379,0.055432
2,A2M,0.524247,0.371010,0.597191,0.781888,0.859964,0.166605,0.919367,0.860522,0.410158,...,0.733399,0.066039,0.487398,0.700757,0.032677,0.595801,0.956441,0.946679,0.980379,0.963082
3,A2ML1,0.418119,0.236624,0.041786,0.370243,0.132408,0.643291,0.378278,0.050163,0.544712,...,0.967032,0.681433,0.872861,0.677861,0.405917,0.925642,1.000000,0.227138,0.177114,0.848094
4,A3GALT2,0.354588,0.935364,0.607998,0.620974,0.454795,0.625648,0.434492,0.389345,0.817776,...,0.978210,0.275538,0.952947,0.955870,0.801143,0.210548,1.000000,0.915732,0.991332,0.870341


In [256]:
metadata = fread('data/SampleInformation_pipeline_abagen.csv', header = True).to_pandas()

In [257]:
metadata.shape

(3466, 15)

In [258]:
metadata.head()

,SampleID,structure_id,slab_num,well_id,slab_type,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,Donor
0,4077-22-594,4077,22,594,CX,PCLa-i,"paracentral lobule, anterior part, right, infe...",37470,87,52,116,5.9,-27.7,49.7,H0351.2001
1,4323-11-2985,4323,11,2985,CX,Cl,"claustrum, right",40517,66,92,63,29.2,17.0,-2.9,H0351.2001
2,4323-18-2801,4323,18,2801,CX,Cl,"claustrum, right",41516,66,81,104,28.2,-22.8,16.8,H0351.2001
3,4440-18-2273,4440,18,2273,CX,LGd,"dorsal lateral geniculate nucleus, left",41473,116,94,101,-24.6,-24.6,1.3,H0351.2001
4,4266-17-2785,4266,17,2785,CX,CA4,"CA4 field, right",41142,63,104,106,31.1,-31.3,-7.3,H0351.2001
